In [ ]:
#gather original df

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

#get plots to show up in jn
%matplotlib inline
#get plots to be an svg when exporting
%config InlineBackend.figure_format = 'svg'

In [ ]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [180407, 180414, 180421, 180428, 180505, 180512, 180519, 180526, 180602, 180609, 180616, 180623]
data = get_data(week_nums)
data.head()

In [ ]:
#cleaning up data

df = data.copy()

df.columns = [column.strip() for column in df.columns]

df['STATION'].apply(lambda x: x.strip())

In [ ]:
#get datetime column
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, format="%m/%d/%Y %H:%M:%S")

#drop dupes

df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=False)

df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

df = df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [ ]:
#sort so prev entries will work
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False))

In [ ]:
#put in prev entries column
df_daily = df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.ENTRIES.first().reset_index().copy()

df_daily[["PREV_DATE", "PREV_ENTRIES"]] = (df_daily.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"].transform(lambda grp: grp.shift(1)))

In [ ]:
#fix daily counts

def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        #print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
        # if current entries is bad, use yesterday's count as proxy
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

df_daily.apply(get_daily_counts, axis=1, max_counter=1000000);

In [ ]:
#subtract prev values to make daily entries column
df_daily["DAILY_ENTRIES"] = df_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
df_daily_sta = df_daily.groupby(['STATION', 'DATE'])['DAILY_ENTRIES'].sum().reset_index().sort_values('DATE')

In [ ]:
dates = df_daily_sta.DATE

riders = df_daily_sta.DAILY_ENTRIES

plt.figure(figsize =(12,6))
plt.plot(dates, riders)
plt.xticks(rotation = 'vertical');

In [ ]:
df_daily_sta.head()

In [ ]:
df_daily.head()